In this notebook, we introduce some basic uses of the agavepy Python library for interacting with the Agave Platform science-as-a-service APIs. The examples primarily draw from the apps service, but the concepts introduced are broadly applicable to all Agave services. In subsequent notebooks, we'll take deeper dives into specific topics such as using agavepy to launch and monitor an Agave job. For more information about Agave, please see the developer site: http://agaveapi.co/

In [10]:
!mkdir -p ~/agave
%cd ~/agave
!pip install --upgrade setvar
!pip install agavepy

/Users/scleveland/AGAVE
Requirement already up-to-date: setvar in /Users/scleveland/anaconda/envs/py36/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 102kB 431kB/s 
    100% |████████████████████████████████| 378kB 3.0MB/s 
    100% |████████████████████████████████| 204kB 4.4MB/s 
    100% |████████████████████████████████| 61kB 7.3MB/s 
    100% |████████████████████████████████| 829kB 1.7MB/s 
  Running setup.py bdist_wheel for agavepy ... done
  Stored in directory: /Users/scleveland/Library/Caches/pip/wheels/84/7a/f6/84d22b0c479e47b7236dcff3d752f4fef3bc188c33a5fc3eb8
  Running setup.py bdist_wheel for Jinja2 ... done
  Stored in directory: /Users/scleveland/Library/Caches/pip/wheels/01/a3/34/1e8ce0a7bfc263c9cc7c87db665fd431105d7be00e289cd69f
  Running setup.py bdist_wheel for backports.ssl-match-hostname ... done


The agavepy library provides a high-level Python binding to the Agave API. The first step is to import the Agave class:

In [81]:
from agavepy.agave import Agave
import re
import os
import sys
from time import sleep
from setvar import *
# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import json


The first thing we need to do to work with the Agave APIs is to define what Agave instance "tenant" we are going to connect to and what credentials to use to do so.  We will assign 'ag' our Agave OAuth connection object for connecting to the UH HPC Tenant using your UH login credentials.

In [52]:
ag = Agave(api_server='https://uhhpctenant.its.hawaii.edu',
username=os.environ['AGAVE_USERNAME'],
password=os.environ['AGAVE_PASSWORD'])

Before we can interact with UH Agave Tenant, we need to instantiate a client session. 

In [6]:
ag.clients.list()

[{'description': '',
  'name': 'agave-togo',
  'consumerKey': 'kYBOf_tsDEk_12y5WBiwteHuXxUa',
  '_links': {'subscriber': {'href': 'https://uhhpctenant.its.hawaii.edu/profiles/v2/seanbc'},
   'self': {'href': 'https://uhhpctenant.its.hawaii.edu/clients/v2/agave-togo'},
   'subscriptions': {'href': 'https://uhhpctenant.its.hawaii.edu/clients/v2/agave-togo/subscriptions/'}},
  'tier': 'Unlimited',
  'callbackUrl': 'https://uphpctenant.its.hawaii.edu/'},
 {'description': '',
  'name': 'anewtestrer',
  'consumerKey': 'FVW5LRld7W2bUZz645UaUgdqn40a',
  '_links': {'subscriber': {'href': 'https://uhhpctenant.its.hawaii.edu/profiles/v2/seanbc'},
   'self': {'href': 'https://uhhpctenant.its.hawaii.edu/clients/v2/anewtestrer'},
   'subscriptions': {'href': 'https://uhhpctenant.its.hawaii.edu/clients/v2/anewtestrer/subscriptions/'}},
  'tier': 'Unlimited',
  'callbackUrl': ''},
 {'description': None,
  'name': 'DefaultApplication',
  'consumerKey': 'AtfPPXtOGv0fO8r8JMh5xF7SrpAa',
  '_links': {'subs

You will observe that you should have no clients currently.
Will use the client constructor by adding the name of the client we want to define. Our login/auth credentials have already been stored in the 'ag' Agave OAuth object instance so we do not need to put them in again. Upon client creation we will have generated an Oauth session for working with the Agave APIs hosted by UH.

We will first delete the client in case we are running this more than once.

In [53]:
ag.clients.delete(clientName='my_client')

{}

In [54]:
ag.clients.create(body={'clientName': 'my_client'})

{'description': '',
 'name': 'my_client',
 'consumerKey': 'fej5rmBI_GJKEBNbJpG9JL9gxKga',
 '_links': {'subscriber': {'href': 'https://uhhpctenant.its.hawaii.edu/profiles/v2/seanbc'},
  'self': {'href': 'https://uhhpctenant.its.hawaii.edu/clients/v2/my_client'},
  'subscriptions': {'href': 'https://uhhpctenant.its.hawaii.edu/clients/v2/my_client/subscriptions/'}},
 'tier': 'Unlimited',
 'consumerSecret': '0ph0QS2nw_1vfIhq5KN3aYlrgS4a',
 'callbackUrl': ''}

<div style="background:#eee;border:1px solid #ccc;padding:5px 10px;"><em>The agavepy library&#39;s Agave class also provides a restore() method for reconstituting previous OAuth sessions. Previous sessions are read from and written to a cache file, /etc/.agpy, so that OAuth sessions persist across iPython sessions. When you authenticated to JupyterHub, the OAuth login was written to the .agpy file. We can therefore use the restore method to create an OAuth client without needing to pass any credentials:</em></div>

<div style="background:#eee;border:1px solid #ccc;padding:5px 10px;"><em>Note that the restore method can take arguments (such as client_name) so that you can restore/manage multiple OAuth sessions. When first getting started on the hub, there is only one session in the cache file, so no arguments are required.</em></div>


If we ever want to inspect the OAuth session being used by our client, we have a few methods available to us. First, we can print the token_info dictionary on the token object:

In [9]:
ag.token.token_info

{u'access_token': u'e4fd3e3bdc761f17efe97d296be43396',
 'created_at': 1523851691,
 'expiration': 1523866091,
 'expires_at': 'Sun Apr 15 22:08:11 2018',
 u'expires_in': 14400,
 u'refresh_token': u'f19b4ef976cc369b625aa84726a534',
 u'scope': u'default',
 u'token_type': u'bearer'}

In [10]:
ag.token.refresh()

u'51a44c8ecf6f702388a7d8252dad42a2'

In [11]:
ag.token.token_info

{u'access_token': u'51a44c8ecf6f702388a7d8252dad42a2',
 'created_at': 1523851698,
 'expiration': 1523866098,
 'expires_at': 'Sun Apr 15 22:08:18 2018',
 u'expires_in': 14400,
 u'refresh_token': u'63c44c32b5fcb4c6bd19f6e17ee833',
 u'scope': u'default',
 u'token_type': u'bearer'}

This shows us both the access and refresh tokens being used. We can also see the end user profile associated with these tokens:

In [40]:
ag.profiles.get()

{u'create_time': u'',
 u'email': u'seanbc@hawaii.edu',
 u'first_name': u'Sean',
 u'full_name': u'Sean B Cleveland',
 u'last_name': u'Cleveland',
 u'mobile_phone': u'',
 u'phone': u'(808) 956-8982',
 u'status': u'',
 u'username': u'seanbc'}

Finally, we can inspect the ag object directly for attributes like api_key, api_secret, api_server, etc.

In [41]:
print ag.api_key, ag.api_secret, ag.api_server

7qgz6sCbDiCfh5fQ60JdcDzujpwa RM021_WqJln0ckgA7MOcNmI8Q_4a https://uhhpctenant.its.hawaii.edu


We are now ready to interact with Agave services using our agavepy client. We can take a quick look at the available top-level methods of our client:

In [45]:
dir(ag)

[u'actors',
 u'admin',
 u'apps',
 u'clients',
 u'files',
 u'jobs',
 u'meta',
 u'monitors',
 u'notifications',
 u'postits',
 u'profiles',
 u'systems',
 u'transforms']

We see there is a top-level method for each of the core science APIs in agave. We will focus on the apps service since it is of broad interest, but much of what we illustrate is generally applicable to all Agave core science APIs.

We can browse a specific collection using the list() method. For example, let's see what apps are available to us:

In [12]:
ag.apps.list()

[{'id': 'seanbc-sandbox-funwave-1.0',
  'name': 'seanbc-sandbox-funwave',
  'version': '1.0',
  'revision': 1,
  'executionSystem': 'hawaii-exec-seanbc',
  'shortDescription': 'Runs a command',
  'isPublic': False,
  'label': 'Runs a command',
  'lastModified': datetime.datetime(2018, 4, 19, 3, 57, 56, tzinfo=tzoffset(None, -18000)),
  '_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/apps/v2/seanbc-sandbox-funwave-1.0'}}},
 {'id': 'deardooley-sandbox-fork-1.0',
  'name': 'deardooley-sandbox-fork',
  'version': '1.0',
  'revision': 1,
  'executionSystem': 'hawaii-exec-deardooley',
  'shortDescription': 'Runs a command',
  'isPublic': False,
  'label': 'Runs a command',
  'lastModified': datetime.datetime(2018, 4, 19, 2, 51, 46, tzinfo=tzoffset(None, -18000)),
  '_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/apps/v2/deardooley-sandbox-fork-1.0'}}},
 {'id': 'seanbc-sandbox-fork-1.0',
  'name': 'seanbc-sandbox-fork',
  'version': '1.0',
  'revision': 2,
  'e

What we see in the output above is a python list representing the JSON object returned from Agave's apps service. It is a list of objects, each of which representing a single app. Let's capture the first app object and inspect it. To do that we can use normal Python list notation:

In [14]:
app = ag.apps.list()[0]

In [19]:
app.__class__

agavepy.agave.AttrDict

We see that the app object is of type agavepy.agave.AttrDict. That's a Python dictionary with some customizations to provide convenience features such as using dot notation for keys/attributes. For example, we see that the app object has an 'id' key. We can access it directly using dot notation:

In [20]:
app.id

'seanbc-sandbox-funwave-1.0'

Equivalently, we can use normal Python dictionary syntax:

app['id']In Agave, the app id is the unique identifier for the application. We'll come back to that in a minute. For now, just know that this example is very typical of responses from agavepy: in general the JSON response object is represented by lists of AttrDicts.

Stepping back for a second, let's explore the apps collection a bit. We can always get a list of operations available for a collection by using the dir(-) method:

In [21]:
dir(ag.apps)

['add',
 'delete',
 'deletePermissions',
 'deletePermissionsForUser',
 'get',
 'getJobSubmissionForm',
 'list',
 'listByName',
 'listByOntologyTerm',
 'listBySystemId',
 'listByTag',
 'listPermissions',
 'listPermissionsForUser',
 'manage',
 'update',
 'updateApplicationPermissions',
 'updatePermissionsForUser']

Also notice that we have tab-completion on these operations. So, if we start typing "ag.apps.l" and then hit tab, Jupyter provides a select box of operations beginning with "l". Try putting the following cell in focus and then hitting the tab key (but don't actually hit enter or try to execute the cell; otherwise you'll get an exception because there's no method called "l"):

In [22]:
ag.apps.l

AttributeError: Resource 'apps' has no operation 'l'

If we would like to get details about a specific object for which we know the unique id, in general we use the get method, passing in the id for the object. Here, we will use an app id we found from the ag.apps.list command.

In [23]:
ag.apps.get(appId=app.id)

{'id': 'seanbc-sandbox-funwave-1.0',
 'name': 'seanbc-sandbox-funwave',
 'icon': None,
 'uuid': '4624972034765549080-242ac118-0001-005',
 'parallelism': 'SERIAL',
 'defaultProcessorsPerNode': 1,
 'defaultMemoryPerNode': 1,
 'defaultNodeCount': 1,
 'defaultMaxRunTime': None,
 'defaultQueue': None,
 'version': '1.0',
 'revision': 1,
 'isPublic': False,
 'helpURI': None,
 'label': 'Runs a command',
 'owner': 'seanbc',
 'shortDescription': 'Runs a command',
 'longDescription': '',
 'tags': [],
 'ontology': [],
 'executionType': 'CLI',
 'executionSystem': 'hawaii-exec-seanbc',
 'deploymentPath': 'funwave-agave-deployment',
 'deploymentSystem': 'hawaii-storage-seanbc',
 'templatePath': 'funwave-wrapper.txt',
 'testPath': 'funwave-test.txt',
 'checkpointable': False,
 'lastModified': datetime.datetime(2018, 4, 19, 3, 57, 56, tzinfo=tzoffset(None, -18000)),
 'modules': [],
 'available': True,
 'inputs': [],
 'parameters': [{'id': 'compress_output',
   'value': {'visible': True,
    'required':

Whoa, that's a lot of information. We aren't going to give a comprehensive introduction to Agave applications in this notebook. Instead we refer you to the official Agave app tutorial on the website: http://agaveapi.co/documentation/tutorials/app-management-tutorial/

However, we will point out a couple of important points. Let's capture that response in an object called full_app:

In [24]:
full_app = ag.apps.get(appId=app.id)

Complex sub-objects of the application such as application inputs and parameters come back as top level attributes. and are represented as lists. The individual elements of the list are represented as AttrDicts. We can see this by exploring our full_app's inputs:

In [29]:
print(full_app.inputs.__class__); print(full_app.inputs[0].__class__); full_app.inputs

<class 'list'>


IndexError: list index out of range

In [33]:
full_app.inputs

[]

Then, if we want the input id, we can use dot notation or dictionary notation just as before:

In [ ]:
full_app.inputs[0].id

You now have the ability to fully explore individual Agave objects returned from agavepy, but what about searching for objects? The Agave platform provides a powerful search feature across most services, and agavepy supports that as well.

Every retrieval operation in agavepy (for example, apps.list) supports a "search" argument. The syntax for the search argument is identical to that described in the Agave documentation: it uses a dot notation combining search terms, values and (optional) operators. The search object itself should be a python dictionary with strings for the keys and values. Formally, each key:value pair in the dictionary adheres to the following form:
term.operator:value
term.operator:value
 
The operator is optional and defaults to equality ('eq'). For example, the following search filters the list of all apps down to just those with the id attribute equal to our app.id:

In [34]:
ag.apps.list(search={'id': app.id})

[{'id': 'seanbc-sandbox-funwave-1.0',
  'name': 'seanbc-sandbox-funwave',
  'version': '1.0',
  'revision': 1,
  'executionSystem': 'hawaii-exec-seanbc',
  'shortDescription': 'Runs a command',
  'isPublic': False,
  'label': 'Runs a command',
  'lastModified': datetime.datetime(2018, 4, 19, 3, 57, 56, tzinfo=tzoffset(None, -18000)),
  '_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/apps/v2/seanbc-sandbox-funwave-1.0'}}}]

Equivalently, we could explicitly set the equality operator:

In [35]:
ag.apps.list(search={'id.eq': app.id})

[{'id': 'seanbc-sandbox-funwave-1.0',
  'name': 'seanbc-sandbox-funwave',
  'version': '1.0',
  'revision': 1,
  'executionSystem': 'hawaii-exec-seanbc',
  'shortDescription': 'Runs a command',
  'isPublic': False,
  'label': 'Runs a command',
  'lastModified': datetime.datetime(2018, 4, 19, 3, 57, 56, tzinfo=tzoffset(None, -18000)),
  '_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/apps/v2/seanbc-sandbox-funwave-1.0'}}}]

Typically, the list of available search terms is identical to the attributes included in the JSON returned when requesting the full resource description. Operators include 'like', 'lt', 'gt', 'lte', 'gte', etc. See the official Agave documentation for the complete list.

Here we retrieve all apps with a name is "like" *funwave:

In [37]:
ag.apps.list(search={'name.like': '*funwave'})

[{'id': 'seanbc-sandbox-funwave-1.0',
  'name': 'seanbc-sandbox-funwave',
  'version': '1.0',
  'revision': 1,
  'executionSystem': 'hawaii-exec-seanbc',
  'shortDescription': 'Runs a command',
  'isPublic': False,
  'label': 'Runs a command',
  'lastModified': datetime.datetime(2018, 4, 19, 3, 57, 56, tzinfo=tzoffset(None, -18000)),
  '_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/apps/v2/seanbc-sandbox-funwave-1.0'}}}]


Two results were returned, both with name "opensees".

You can include multiple search expressions in the form of additional key:value pairs to build a more restrictive query. Here we restrict the result to opensees apps with revision at least 25:

In [42]:
ag.apps.list(search={'name.like': '*funwave', 'revision.gte': 1})

[{'id': 'seanbc-sandbox-funwave-1.0',
  'name': 'seanbc-sandbox-funwave',
  'version': '1.0',
  'revision': 1,
  'executionSystem': 'hawaii-exec-seanbc',
  'shortDescription': 'Runs a command',
  'isPublic': False,
  'label': 'Runs a command',
  'lastModified': datetime.datetime(2018, 4, 19, 3, 57, 56, tzinfo=tzoffset(None, -18000)),
  '_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/apps/v2/seanbc-sandbox-funwave-1.0'}}}]

We hope this gives you enough general information to begin exploring the Agave services using agavepy on your own. In subsequent notebooks, we'll take deeper dives into specific topics such as using agavepy to launch and monitor an Agave job.

In [4]:
ag.systems.list()

[{'_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-lustre-storage'}},
  'default': False,
  'description': 'lustres on uh hpc',
  'id': 'uhhpc-lustre-storage',
  'lastUpdated': '2018-04-17T19:24:10.000-05:00',
  'name': 'uhhpc-lustre-storage',
  'public': False,
  'status': 'UP',
  'type': 'STORAGE'},
 {'_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/execute.uhhpc1.its.hawaii.edu'}},
  'default': False,
  'description': 'Execution system using ssh to submit jobs to the UH ITS HPC.',
  'id': 'execute.uhhpc1.its.hawaii.edu',
  'lastUpdated': '2018-04-16T18:48:00.000-05:00',
  'name': 'UH ITS HPC SSH Execution Host',
  'public': False,
  'status': 'UP',
  'type': 'EXECUTION'},
 {'_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uh-hpc-lustre'}},
  'default': True,
  'description': 'UH HPC Lustre Storage',
  'id': 'uh-hpc-lustre',
  'lastUpdated': '2018-04-16T13:03:00.000-05:00',
  'name': 'UH HPC Lustre',


In [55]:
storage_system_json ={
  "type": "STORAGE",
  "storage": {
    "protocol": "SFTP",
    "proxyTunnel": "NO",
    "host": "uhhpc1.its.hawaii.edu",
    "port": 22,
    "rootDir": "/lus/scratch/"+os.environ['AGAVE_USERNAME'],
    "homeDir": "/",
    "auth": {
      "type": "PASSWORD",
      "username": os.environ['AGAVE_USERNAME'],
      "password": os.environ['AGAVE_PASSWORD'],
    }
  },
  "id": "uhhpc-lustre-storage-"+os.environ['AGAVE_USERNAME'],
  "name": "uhhpc-lustre-storage-"+os.environ['AGAVE_USERNAME'],
  "status": "UP",
  "description": "lustres on uh hpc"
}

In [56]:
ag.systems.add(body=json.dumps(storage_system_json))

{'owner': 'seanbc',
 '_links': {'owner': {'href': 'https://uhhpctenant.its.hawaii.edu/profiles/v2/seanbc'},
  'metadata': {'href': 'https://uhhpctenant.its.hawaii.edu/meta/v2/data/?q=%7B%22associationIds%22%3A%223316049078201094632-242ac1112-0001-006%22%7D'},
  'credentials': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-lustre-storage-seanbc/credentials'},
  'roles': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-lustre-storage-seanbc/roles'},
  'self': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-lustre-storage-seanbc'}},
 'available': True,
 'description': 'lustres on uh hpc',
 'storage': {'proxy': None,
  'protocol': 'SFTP',
  'mirror': False,
  'port': 22,
  'auth': {'type': 'PASSWORD'},
  'publicAppsDir': None,
  'host': 'uhhpc1.its.hawaii.edu',
  'rootDir': '/lus/scratch/seanbc',
  'homeDir': '/'},
 'type': 'STORAGE',
 'uuid': '3316049078201094632-242ac1112-0001-006',
 'revision': 1,
 'site': None,
 'default': False,
 'public': Fal

In [67]:
execution_system_json = {
  "maxSystemJobs": 50,
  "workDir": "",
  "scratchDir": "/lus/scratch/"+os.environ['AGAVE_USERNAME']+"/",
  "type": "EXECUTION",
  "id": "uhhpc-execution-system-"+os.environ['AGAVE_USERNAME'],
  "description": "Execution system using ssh to submit jobs to the UH ITS HPC. By default, it uses the Sandbox queue.",
  "name": "UH ITS HPC SSH Execution Host",
  "login": {
    "port": 22,
    "protocol": "SSH",
    "host": "uhhpc1.its.hawaii.edu",
    "auth": {
      "type": "PASSWORD",
      "username": os.environ['AGAVE_USERNAME'],
      "password": os.environ['AGAVE_PASSWORD'],
    }
  },
  "maxSystemJobsPerUser": 10,
  "site": "hawaii.edu",
  "status": "UP",
  "scheduler": "SLURM",
  "executionType": "HPC",
  "queues": [{
    "maxProcessorsPerNode": 20,
    "default": False,
    "maxMemoryPerNode": "120GB",
    "name": "community.q",
    "maxRequestedTime": "72:00:00",
    "maxJobs": 25,
    "maxNodes": 5,
    "maxUserJobs": 5
  }, {
    "maxProcessorsPerNode": 20,
    "default": False,
    "maxMemoryPerNode": "120GB",
    "name": "kill.q",
    "maxRequestedTime": "72:00:00",
    "maxJobs": 25,
    "maxNodes": 5,
    "maxUserJobs": 5
  }, {
    "maxProcessorsPerNode": 20,
    "default": False,
    "maxMemoryPerNode": "120GB",
    "name": "sb.q",
    "maxRequestedTime": "01:00:00",
    "maxJobs": 25,
    "maxNodes": 2,
    "maxUserJobs": 5
  }, {
    "maxProcessorsPerNode": 20,
    "default": True,
    "maxMemoryPerNode": "120GB",
    "name": "workshop",
    "maxRequestedTime": "00:01:00",
    "maxJobs": 25,
    "maxNodes": 2,
    "maxUserJobs": 2
  }, {
    "maxProcessorsPerNode": 20,
    "default": False,
    "maxMemoryPerNode": "120GB",
    "name": "exclusive.q",
    "maxRequestedTime": "72:00:00",
    "maxJobs": 25,
    "maxNodes": 5,
    "maxUserJobs": 5
  }],
  "public": False,
  "storage": {
    "port": 22,
    "homeDir": "/home/"+os.environ['AGAVE_USERNAME'],
    "protocol": "SFTP",
    "host": "uhhpc1.its.hawaii.edu",
    "rootDir": "/",
    "auth": {
      "type": "PASSWORD",
      "username": os.environ['AGAVE_USERNAME'],
      "password": os.environ['AGAVE_PASSWORD'],
    }
  }
}

In [68]:
ag.systems.add(body=json.dumps(execution_system_json))

{'maxSystemJobs': 50,
 '_links': {'owner': {'href': 'https://uhhpctenant.its.hawaii.edu/profiles/v2/seanbc'},
  'metadata': {'href': 'https://uhhpctenant.its.hawaii.edu/meta/v2/data/?q=%7B%22associationIds%22%3A%22709741008061984280-242ac1112-0001-006%22%7D'},
  'credentials': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-execution-system-seanbc/credentials'},
  'roles': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-execution-system-seanbc/roles'},
  'self': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-execution-system-seanbc'},
  'history': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-execution-system-seanbc/history'}},
 'executionType': 'HPC',
 'available': True,
 'description': 'Execution system using ssh to submit jobs to the UH ITS HPC. By default, it uses the Sandbox queue.',
 'storage': {'proxy': None,
  'protocol': 'SFTP',
  'mirror': False,
  'port': 22,
  'auth': {'type': 'PASSWORD'},
  'host': 'uhhpc1.its.hawa

In [69]:
ag.systems.list()

[{'id': 'uhhpc-execution-system-seanbc',
  'name': 'UH ITS HPC SSH Execution Host',
  'type': 'EXECUTION',
  'description': 'Execution system using ssh to submit jobs to the UH ITS HPC. By default, it uses the Sandbox queue.',
  'status': 'UP',
  'public': False,
  'lastUpdated': '2018-04-19T11:44:44.000-05:00',
  'default': False,
  '_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-execution-system-seanbc'}}},
 {'id': 'uhhpc-lustre-storage-seanbc',
  'name': 'uhhpc-lustre-storage-seanbc',
  'type': 'STORAGE',
  'description': 'lustres on uh hpc',
  'status': 'UP',
  'public': False,
  'lastUpdated': '2018-04-19T11:39:08.000-05:00',
  'default': False,
  '_links': {'self': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-lustre-storage-seanbc'}}},
 {'id': 'hawaii-exec-deardooley',
  'name': 'sandbox (jovyan)',
  'type': 'EXECUTION',
  'description': 'The sandbox computer',
  'status': 'UP',
  'public': False,
  'lastUpdated': '2018-04-19T02:4

In [75]:
system = ag.systems.list(search={'id': 'uhhpc-lustre-storage-'+os.environ['AGAVE_USERNAME']})[0]

In [76]:
system.id

'uhhpc-lustre-storage-seanbc'

In [77]:
#list the files in our storage system
ag.files.list(systemId=system.id,filePath='/')

HTTPError: 502 Server Error: Bad Gateway for url: https://uhhpctenant.its.hawaii.edu/files/v2/listings/system/uhhpc-lustre-storage-seanbc//

In [26]:
#Create directory on storage system
ag.files.manage(body="{'action':'mkdir','path':'test-folders/one'}",systemId=system.id,filePath="/")


{'_links': {'history': {'href': 'https://uhhpctenant.its.hawaii.edu/files/v2/history/system/uhhpc-lustre-storage//test-folders/one'},
  'profile': {'href': 'https://uhhpctenant.its.hawaii.edu/profiles/v2/seanbc'},
  'self': {'href': 'https://uhhpctenant.its.hawaii.edu/files/v2/media/system/uhhpc-lustre-storage//test-folders/one'},
  'system': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-lustre-storage'}},
 'internalUsername': None,
 'lastModified': '2018-04-18T02:54:51.352-05:00',
 'name': 'one',
 'nativeFormat': 'dir',
 'owner': 'seanbc',
 'path': 'test-folders/one',
 'source': None,
 'status': 'TRANSFORMING_COMPLETED',
 'systemId': 'uhhpc-lustre-storage',
 'uuid': '4173046298764832280-242ac113-0001-002'}

In [27]:
ag.files.list(systemId=system.id,filePath="/test-folders/one")

[{'_links': {'history': {'href': 'https://uhhpctenant.its.hawaii.edu/files/v2/history/system/uhhpc-lustre-storage//test-folders/one'},
   'metadata': {'href': 'https://uhhpctenant.its.hawaii.edu/meta/v2/data?q=%7B%22associationIds%22%3A%224173046298764832280-242ac113-0001-002%22%7D'},
   'self': {'href': 'https://uhhpctenant.its.hawaii.edu/files/v2/media/system/uhhpc-lustre-storage//test-folders/one'},
   'system': {'href': 'https://uhhpctenant.its.hawaii.edu/systems/v2/uhhpc-lustre-storage'}},
  'format': 'folder',
  'lastModified': datetime.datetime(2018, 4, 18, 2, 56, 3, tzinfo=tzoffset(None, -18000)),
  'length': 4096,
  'mimeType': 'text/directory',
  'name': '.',
  'path': '/test-folders/one',
  'permissions': 'ALL',
  'system': 'uhhpc-lustre-storage',
  'type': 'dir'}]



#### For some more information see https://agavepy.readthedocs.io/en/latest/ which is the documentation for the `agavepy` module.
